<a href="https://colab.research.google.com/github/sezinbiner/ClassificationOfCovidXray/blob/main/deneme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
import cv2
import os

DATADIR = "/content/gdrive/MyDrive/kaggle/COVID-19_Radiography_Dataset" 
classes = ["COVID","Normal"] 

X = []
y = []
i = -1
for class_name in classes: 
  i += 1
  print(class_name)
  path = os.path.join(DATADIR,class_name)
  print(path)
  os.chdir(path) 
  images = os.listdir(path)[:1000]
  for im in images:
      try:
        print(im)
        img = cv2.imread(im, cv2.IMREAD_COLOR) 
        img = cv2.resize(img, (224,224))
        X.append(img) 
        y.append(classes.index(class_name))   
        print(len(X))  
      except Exception as e:
          pass

COVID
/content/gdrive/MyDrive/kaggle/COVID-19_Radiography_Dataset/COVID
COVID-3362.png
1
COVID-3357.png
2
COVID-3348.png
3
COVID-3337.png
4
COVID-334.png
5
COVID-3358.png
6
COVID-3359.png
7
COVID-3340.png
8
COVID-3341.png
9
COVID-3332.png
10
COVID-3355.png
11
COVID-336.png
12
COVID-3371.png
13
COVID-3379.png
14
COVID-3367.png
15
COVID-3369.png
16
COVID-3368.png
17
COVID-3366.png
18
COVID-3378.png
19
COVID-3373.png
20
COVID-3377.png
21
COVID-3376.png
22
COVID-3374.png
23
COVID-3365.png
24
COVID-337.png
25
COVID-3372.png
26
COVID-3370.png
27
COVID-3364.png
28
COVID-3375.png
29
COVID-3390.png
30
COVID-339.png
31
COVID-3387.png
32
COVID-3383.png
33
COVID-3393.png
34
COVID-3382.png
35
COVID-3388.png
36
COVID-3381.png
37
COVID-3392.png
38
COVID-3386.png
39
COVID-3384.png
40
COVID-3389.png
41
COVID-3385.png
42
COVID-338.png
43
COVID-3391.png
44
COVID-3380.png
45
COVID-3406.png
46
COVID-3401.png
47
COVID-3402.png
48
COVID-34.png
49
COVID-3404.png
50
COVID-3395.png
51
COVID-341.png
52
COVID-339

In [18]:
import numpy as np
x = np.array(X)
y = np.array(y)
x = x.astype(np.float32)  
x = x/255
print(len(x))
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.10,shuffle=True)
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.25,shuffle=True)
print(len(x_train))
print(len(x_val))
print(len(x_test))
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)
y_val_one_hot = to_categorical(y_val)

2000
1350
450
200


In [19]:
from tensorflow.keras import layers
from tensorflow.keras import Model
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2021-05-08 14:58:51--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 74.125.20.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   262MB/s    in 0.3s    

2021-05-08 14:58:52 (262 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [20]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(
    input_shape=(224, 224, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)

In [21]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [22]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

last layer output shape: (None, 12, 12, 768)


In [23]:
from tensorflow.keras.optimizers import RMSprop
# Flatten the output layer to 1 dimension
a = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
a = layers.Dense(1024, activation='relu')(a)
# Add a dropout rate of 0.2
a = layers.Dropout(0.2)(a)
# Add a final sigmoid layer for classification
a = layers.Dense(1, activation='sigmoid')(a)

# Configure and compile the model
model = Model(pre_trained_model.input, a)
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.0001),
              metrics=['acc'])

In [ ]:
model.summary()

In [29]:
model.fit(x, y, epochs=15, validation_data=(x_val, y_val))

Epoch 1/15
63/63 [==============================] - 6s 90ms/step - loss: 0.0405 - acc: 0.9900 - val_loss: 0.0697 - val_acc: 0.9711
Epoch 2/15
63/63 [==============================] - 5s 75ms/step - loss: 0.0273 - acc: 0.9925 - val_loss: 4.5145e-04 - val_acc: 1.0000
Epoch 3/15
63/63 [==============================] - 5s 75ms/step - loss: 0.0698 - acc: 0.9870 - val_loss: 4.5636e-04 - val_acc: 1.0000
Epoch 4/15
63/63 [==============================] - 5s 75ms/step - loss: 0.0136 - acc: 0.9960 - val_loss: 5.2381e-04 - val_acc: 1.0000
Epoch 5/15
63/63 [==============================] - 5s 75ms/step - loss: 0.0010 - acc: 1.0000 - val_loss: 9.1222e-05 - val_acc: 1.0000
Epoch 6/15
63/63 [==============================] - 5s 76ms/step - loss: 0.0518 - acc: 0.9890 - val_loss: 4.2484e-05 - val_acc: 1.0000
Epoch 7/15
63/63 [==============================] - 5s 75ms/step - loss: 6.9302e-05 - acc: 1.0000 - val_loss: 3.5928e-05 - val_acc: 1.0000
Epoch 8/15
63/63 [==============================] - 5s 

In [30]:
print(model.metrics_names)
from sklearn import metrics
y_pred_one_hot = model.predict(x_test)
y_pred_labels = np.argmax(y_pred_one_hot, axis = 1)
y_true_labels = np.argmax(y_test_one_hot,axis=1)

['loss', 'acc']


In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_true_labels, y_pred_labels)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_true_labels, y_pred_labels,pos_label='positive', average='micro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_true_labels, y_pred_labels, average='micro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')
print('F1 score: %f' % f1)


Accuracy: 0.445000
Precision: 0.445000
Recall: 0.445000
F1 score: 0.274083


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
